In [7]:
import numpy as np
import cProfile
import matplotlib.pyplot as plt
from time import time

In [5]:
def fox(exponent: int) -> float:

    np.random.seed(69)  # set the seed for the random numbers generator
                        # so that the same numbers are generated in each process 

    MATRIX_SIZE = 2**exponent
    # generate two random matrix of size MATRIX_SIZE
    matrix_A    = np.random.randint(1000, 2000, (MATRIX_SIZE, MATRIX_SIZE)) 
    matrix_B    = np.random.randint(1000, 2000, (MATRIX_SIZE, MATRIX_SIZE)) 
    # initialize the matrix C with zeros
    matrix_C    = np.zeros((MATRIX_SIZE, MATRIX_SIZE), dtype=int)
    start_time = time()

    for row_i in range(MATRIX_SIZE): 
        # Each process calculates a row of the matrix C
        # The process with rank i calculates the row i of the matrix C
        # The row i of the matrix C is the sum of the rows of the matrix A multiplied by the matrix B

        for i in range(MATRIX_SIZE):
            # a[row_i, row_i] gets shifted to the right by i positions
            # and b[row_i] gets shifted to the bottom by i positions
            col = (row_i + i) % MATRIX_SIZE
            matrix_C[row_i] += matrix_A[row_i, col] * matrix_B[col] 

        # The rows of the matrix C are distributed among the processes using the MPI_Allreduce function
        # The MPI_Allreduce function sums the rows of the matrix C calculated by each process

    end_time = time() - start_time
    np.savetxt('arrayA.txt', matrix_A, fmt='%d')
    np.savetxt('arrayB.txt', matrix_B, fmt='%d')
    np.savetxt('result.txt', matrix_C, fmt='%d')
    return end_time

In [6]:
fox(4)

0.0010008811950683594

In [ ]:

def graphs():
    times_MPI = {}
    times_SEC = {}
    for exponent in range(6, 9):
        times_MPI[exponent] = np.zeros(5)
        times_SEC[exponent] = np.zeros(5)
        for i in range(5):
            times_MPI[exponent][i] = fox(exponent, True)
            times_SEC[exponent][i] = fox(exponent, False)
    
    fig, ax = plt.subplots()
    y = [np.mean(times_MPI[exponent]) for exponent in times_MPI]
    x = [exponent for exponent in times_MPI]
    x_labels = [str(2**exponent) for exponent in times_MPI]
    plt.figure(figsize=(10, 5))
    plt.title('Tiempo de ejecución del algoritmo Fox\nMatrices de números enteros.')
    plt.xlabel('Orden de la Matriz')
    plt.xticks(x, x_labels)
    plt.yticks([]) 
    bars = plt.bar(x, y, label='MPI')

    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:.5}s', va='bottom', ha='center')  # va: vertical alignment

    plt.show()

In [ ]:
graphs()

TypeError: unsupported operand type(s) for %: 'int' and 'NoneType'